In [1]:
import requests

In [2]:
# Open and read file
with open ('../../../Auth_Keys/graphql_api_auth.txt') as file:
    token = file.read()

In [3]:
# Specify the Authorization code
headers = {"Authorization": 'Bearer ' + token}

In [4]:
def run_query(query): # A simple function to use requests.post to make the API call. Note the json= section.
    request = requests.post('https://api.github.com/graphql', json={'query': query}, headers=headers)
    if request.status_code == 200:
        return request.json()
    else:
        raise Exception("Query failed to run by returning code of {}. {}".format(request.status_code, query))

In [5]:
# https://blog.codeship.com/an-introduction-to-graphql-via-the-github-api/
# https://developer.github.com/v4/explorer/ # To enter the graphql exploer
# https://help.github.com/en/articles/searching-issues-and-pull-requests # For searches

In [29]:
# Query all info
query = '''
{
  repositoryOwner(login: "pandas-dev") {
    id
    login
    repository(name: "pandas") {
      id
      name
      createdAt
      updatedAt
      description
      licenseInfo {
        spdxId
      }
      forks(first:1) {
        totalCount
        edges {
          node {
            createdAt
            updatedAt
            name
          }
        }
      }
      watchers(first:1) {
        totalCount
        edges {
          node {
            createdAt
            updatedAt
            login
            name
            company
          }
        }
      }
    }
  }
}
'''

In [ ]:
# Query releases
query = '''
{
  repositoryOwner(login: "pandas-dev") {
    id
    login
    repository(name: "pandas") {
      id
      name
      createdAt
      updatedAt
      description
      licenseInfo {
        spdxId
      }
      releases(first:10) {
        totalCount
        edges {
          node {
            name
            description
            createdAt
            publishedAt
          }
        }
      }
    }
  }
}
'''

In [ ]:
# Query star gazers
query = '''
{
  repositoryOwner(login: "pandas-dev") {
    id
    login
    repository(name: "pandas") {
      id
      name
      createdAt
      updatedAt
      description
      licenseInfo {
        spdxId
      }
      stargazers(first:1) {
        totalCount
        pageInfo {
          endCursor
          hasNextPage
        }
        edges {
          starredAt
          node {
            createdAt
            login
            company
          }
        }
      }
    }
  }
}
'''

In [2]:
result = run_query(query)
result

NameError: name 'run_query' is not defined

In [17]:
# Query Pull Requests
# Change the number of pull request to retrieve the dates
# https://stackoverflow.com/questions/51972020/github-graphql-query-count-pr-reviews-by-user-for-a-given-month
# https://help.github.com/en/articles/searching-issues-and-pull-requests
# https://help.github.com/en/articles/understanding-the-search-syntax#query-for-dates
query = '''
{
  search(first:100, query:"repo:pandas-dev/pandas created:2010-08-24..2010-08-25 type:issue", type:ISSUE) {
    nodes {
      ... on Issues {
        createdAt
        updatedAt
        title
        mergedBy {
          login
        }
        author {
          login
          ... on User {
            company
          }
        }
        authorAssociation
        files {
          totalCount
        }
        state
        resourcePath
        bodyText
        comments(first:100) {
          totalCount
          edges {
            node {
              author {
                login
                ... on User {
                  company
                }
              }
              authorAssociation
              bodyText
              reactions(first:100) {
                edges {
                  node {
                    content
                  }
                }
              }
            }
          }
        }
        reviewThreads(first:100) {
          edges {
            node {
              comments(first:100) {
                edges {
                  node {
                    bodyText
                  }
                }
              }
            }
          }
        }
      }
    }
  }
}
'''

In [26]:
# Query Pull Request_2
query = '''
{
  search(first:100, query:"repo:pandas-dev/pandas created:2010-08-24..2010-08-25 type:pr", type:ISSUE) {
    nodes {
      ... on PullRequest {
        createdAt
        updatedAt
        title
        mergedBy {
          login
        }
        author {
          login
          ... on User {
            company
          }
        }
        authorAssociation
        files {
          totalCount
        }
        state
        resourcePath
        bodyText
        comments(first:20) {
          totalCount
          edges {
            node {
              author {
                login
                ... on User {
                  company
                }
              }
              authorAssociation
              bodyText
            }
          }
        }
      }
    }
  }
}
'''

In [27]:
result = run_query(query)
result

Exception: Query failed to run by returning code of 502. 
{
  search(first: 100, query: "repo:pandas-dev/pandas created:2018-08-24..2018-08-31 type:pr", type: ISSUE) {
    nodes {
      ... on PullRequest {
        id
        createdAt
        updatedAt
        title
        mergedBy {
          login
        }
        author {
          login
          ... on User {
            company
          }
        }
        authorAssociation
        files {
          totalCount
        }
        state
        resourcePath
        bodyText
        comments(first: 20) {
          totalCount
          edges {
            node {
              id
              author {
                login
                ... on User {
                  contributionsCollection {
                    hasAnyContributions
                  }
                  company
                }
              }
              authorAssociation
            }
          }
        }
      }
    }
  }
}


# Commits

In [ ]:
# Use end cursor and after parameter to search all commmits
query ='''
{
  repository(owner: "pandas-dev", name: "pandas") {
    ref(qualifiedName: "master") {
      target {
        ... on Commit {
          history(first: 10) {
            pageInfo {
              hasNextPage
              endCursor
            }
            edges {
              node {
                author {
                  user {
                    login
                  }
                }
                oid
                messageHeadline
                committedDate
                committer {
                  user {
                    login
                    company
                  }
                }
                status {
                  id
                  state
                }
                associatedPullRequests(first:5) {
                  edges {
                    node {
                      id
                      title
                      author {
                        login
                      }
                      authorAssociation
                      createdAt
                      updatedAt
                      closedAt
                      number
                      state
                    }
                  }
                }
              }
            }
          }
        }
      }
    }
  }
}
'''

# Randomized Search of Repos using DatabaseId

In [ ]:
query = '''
{
  search(query: "is:public created:2019-08-26..2019-08-26", type: REPOSITORY, first:100) {
    repositoryCount
    edges {
      node {
        ... on Repository {
          id
          nameWithOwner
          name
          databaseId
          createdAt
        }
      }
    }
  }
}
'''

In [24]:
from gql import gql, Client

In [ ]:
client = Client(schema=schema)
query = gql('''
{
  hello
}
''')

client.execute(query)

In [54]:
# Query Issues
# Change the number of issues to retrieve the dates
query = '''
{
    repositoryOwner(login: "pandas-dev") {
        repository(name: "pandas") {
            issues(last:3) {
                edges {
                    node {
                        id
                        createdAt
                        closedAt
                        title
                        bodyText
                    }
                }
            } 
        }
    }
}
'''

In [ ]:
# Query release
# Change the number of issues to retrieve the dates

In [55]:
result = run_query(query)
result

In [57]:
query = '''
{
  organization(login: "pandas-dev") {
    name
    description
    url
    gist
  }
}
'''

In [28]:
# Check limit
query = '''
{
  viewer {
    login
  }
  rateLimit {
    limit
    cost
    remaining
    resetAt
  }
}
'''
result = run_query(query)
result

{'data': {'viewer': {'login': 'dustiny5'},
  'rateLimit': {'limit': 5000,
   'cost': 1,
   'remaining': 4999,
   'resetAt': '2019-08-22T22:06:15Z'}}}